In [1]:
!pip -q install transformers>=4.38.0 trl>=0.8.0 peft>=0.9.0 bitsandbytes evaluate

import evaluate
from datasets import load_dataset
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model
import torch
from tqdm.auto import tqdm

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
libcugraph-cu12 25.6.0 requires libraft-cu12==25.6.*, but you have libraft-cu12 25.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.
pylibcugraph-cu12 25.6.0 requires pylibraft-cu12==25.6.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 25.6.

2025-11-12 14:06:31.036038: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762956391.273378      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762956391.333744      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
checkpoint = 'google-t5/t5-3b'
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
MAX_LENGTH = model.config.task_specific_params['summarization']['max_length']
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/11.4G [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [3]:
dataset = load_dataset('cnn_dailymail', '3.0.0').remove_columns('id')

dataset['train'] = dataset['train'].select(range(10000))
dataset['validation'] = dataset['validation'].select(range(1000))

def tokenize_function(examples):
    inputs = tokenizer(
        ['summarize: ' + text for text in examples['article']],
        max_length=MAX_LENGTH,
        truncation=True
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            [example[0] for example in examples['highlights']],
            max_length=MAX_LENGTH,
            truncation=True
        )

    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': labels['input_ids']
    }

not_preprocessed_val_data = dataset['validation']
dataset = dataset.map(tokenize_function, batched=True, batch_size=1000)
dataset = dataset.remove_columns(['article', 'highlights'])
train_dataset, val_dataset = dataset['train'], dataset['validation']

README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

3.0.0/validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [5]:
def generate_response(promt):
    inputs = tokenizer(
        promt,
        max_length=MAX_LENGTH,
        truncation=True,
        padding='longest',
        return_tensors='pt'
    )

    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=MAX_LENGTH,
            early_stopping=True,
            num_beams=4,
            # temperature=0.7,
            # top_p=0.9,
            length_penalty=1.1,
            no_repeat_ngram_size=3
        )

    texts = [
        tokenizer.decode(output, skip_special_tokens=True)
        for output in outputs
    ]

    return texts

rouge = evaluate.load('rouge')
def evaluate_model(model, tokenizer, val_data, batch_size=8):
    predictions = []

    for i in tqdm(range(0, len(val_data), batch_size)):
        batch = val_data.select(range(i, min(i+batch_size, len(val_data))))
        batch_predictions = generate_response(['summarize: ' + item['article'] for item in batch])
        predictions.extend(batch_predictions)

    references = [item['highlights'] for item in val_data]

    return rouge.compute(predictions=predictions, references=references)

In [6]:
model.eval()
model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=4096, bias=False)
              (k): Linear(in_features=1024, out_features=4096, bias=False)
              (v): Linear(in_features=1024, out_features=4096, bias=False)
              (o): Linear(in_features=4096, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=16384, bias=False)
              (wo): Linear(in_features=16384, out_features=1024, bias=False)
              

In [7]:
promts = ['summarize: One side of the armed conflicts is composed mainly of the Sudanese military and the Janjaweed, a Sudanese militia group recruited mostly from the Afro-Arab Abbala tribes of the northern Rizeigat region in Sudan.']
generated_texts = generate_response(promts)
print(generated_texts[0])

Sudanese military and the Janjaweed, a sudanese militia group recruited mostly from the Afro-Arab Abbala tribes of the northern Rizeigat region in Sudan.


In [8]:
import gc; gc.collect()
torch.cuda.empty_cache()

In [9]:
scores = evaluate_model(model, tokenizer, not_preprocessed_val_data.select(range(30)), batch_size=1)
print(scores)

  0%|          | 0/30 [00:00<?, ?it/s]

{'rouge1': 0.40551666036183737, 'rouge2': 0.22058954375166137, 'rougeL': 0.31871927831710123, 'rougeLsum': 0.3651192863596151}
